In [ ]:
!pip install torch torchvision scikit-learn tqdm

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from tqdm import tqdm
import numpy as np

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [4]:
DATA_DIR = "dataset_split/eyes"   

In [5]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [6]:
train_ds = datasets.ImageFolder(f"{DATA_DIR}/train", transform=train_transforms)
val_ds   = datasets.ImageFolder(f"{DATA_DIR}/val", transform=val_test_transforms)
test_ds  = datasets.ImageFolder(f"{DATA_DIR}/test", transform=val_test_transforms)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=32, shuffle=False)

print("Classes:", train_ds.classes)

Classes: ['closed', 'open']


In [ ]:
model = models.mobilenet_v3_large(weights="IMAGENET1K_V1")

In [8]:
model.classifier[3] = nn.Linear(
    model.classifier[3].in_features, 1
)

model = model.to(device)


In [9]:
for param in model.features.parameters():
    param.requires_grad = False

In [10]:
criterion = nn.BCEWithLogitsLoss()

optimizer = optim.Adam(
    model.classifier.parameters(),
    lr=1e-4
)

In [11]:
def train_one_epoch(model, loader):
    model.train()
    total_loss = 0

    for images, labels in tqdm(loader):
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)

In [12]:
def evaluate(model, loader):
    model.eval()
    y_true, y_pred, y_prob = [], [], []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)

            probs = torch.sigmoid(outputs).cpu().numpy()
            preds = (probs > 0.5).astype(int)

            y_true.extend(labels.numpy())
            y_pred.extend(preds)
            y_prob.extend(probs)

    return np.array(y_true), np.array(y_pred), np.array(y_prob)

In [13]:
EPOCHS = 10

for epoch in range(EPOCHS):
    loss = train_one_epoch(model, train_loader)
    y_true, y_pred, _ = evaluate(model, val_loader)

    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    print("Train Loss:", round(loss, 4))
    print(classification_report(y_true, y_pred, target_names=train_ds.classes))

100%|██████████| 32/32 [00:16<00:00,  1.93it/s]



Epoch 1/10
Train Loss: 0.4893
              precision    recall  f1-score   support

      closed       0.94      0.87      0.90       108
        open       0.88      0.94      0.91       108

    accuracy                           0.91       216
   macro avg       0.91      0.91      0.91       216
weighted avg       0.91      0.91      0.91       216



100%|██████████| 32/32 [00:16<00:00,  2.00it/s]



Epoch 2/10
Train Loss: 0.241
              precision    recall  f1-score   support

      closed       0.96      0.94      0.95       108
        open       0.95      0.96      0.95       108

    accuracy                           0.95       216
   macro avg       0.95      0.95      0.95       216
weighted avg       0.95      0.95      0.95       216



100%|██████████| 32/32 [00:15<00:00,  2.08it/s]


...


Epoch 8/10
Train Loss: 0.0762
              precision    recall  f1-score   support

      closed       0.98      0.98      0.98       108
        open       0.98      0.98      0.98       108

    accuracy                           0.98       216
   macro avg       0.98      0.98      0.98       216
weighted avg       0.98      0.98      0.98       216



100%|██████████| 32/32 [00:15<00:00,  2.02it/s]



Epoch 9/10
Train Loss: 0.0686
              precision    recall  f1-score   support

      closed       0.98      0.99      0.99       108
        open       0.99      0.98      0.99       108

    accuracy                           0.99       216
   macro avg       0.99      0.99      0.99       216
weighted avg       0.99      0.99      0.99       216



100%|██████████| 32/32 [00:15<00:00,  2.05it/s]



Epoch 10/10
Train Loss: 0.0559
              precision    recall  f1-score   support

      closed       0.98      0.98      0.98       108
        open       0.98      0.98      0.98       108

    accuracy                           0.98       216
   macro avg       0.98      0.98      0.98       216
weighted avg       0.98      0.98      0.98       216



In [14]:
for param in model.features[-2:].parameters():
    param.requires_grad = True

optimizer = optim.Adam(model.parameters(), lr=1e-5)


In [15]:
for epoch in range(5):
    loss = train_one_epoch(model, train_loader)
    y_true, y_pred, _ = evaluate(model, val_loader)

    print(f"\nFine-tune Epoch {epoch+1}")
    print("Loss:", round(loss, 4))
    print(classification_report(y_true, y_pred))


100%|██████████| 32/32 [00:17<00:00,  1.83it/s]



Fine-tune Epoch 1
Loss: 0.0541
              precision    recall  f1-score   support

           0       0.98      0.97      0.98       108
           1       0.97      0.98      0.98       108

    accuracy                           0.98       216
   macro avg       0.98      0.98      0.98       216
weighted avg       0.98      0.98      0.98       216



100%|██████████| 32/32 [00:17<00:00,  1.84it/s]



Fine-tune Epoch 2
Loss: 0.0584
              precision    recall  f1-score   support

           0       0.98      0.97      0.98       108
           1       0.97      0.98      0.98       108

    accuracy                           0.98       216
   macro avg       0.98      0.98      0.98       216
weighted avg       0.98      0.98      0.98       216



100%|██████████| 32/32 [00:16<00:00,  1.92it/s]



Fine-tune Epoch 3
Loss: 0.0688
              precision    recall  f1-score   support

           0       0.98      0.97      0.98       108
           1       0.97      0.98      0.98       108

    accuracy                           0.98       216
   macro avg       0.98      0.98      0.98       216
weighted avg       0.98      0.98      0.98       216



100%|██████████| 32/32 [00:16<00:00,  1.96it/s]



Fine-tune Epoch 4
Loss: 0.0517
              precision    recall  f1-score   support

           0       0.98      0.97      0.98       108
           1       0.97      0.98      0.98       108

    accuracy                           0.98       216
   macro avg       0.98      0.98      0.98       216
weighted avg       0.98      0.98      0.98       216



100%|██████████| 32/32 [00:17<00:00,  1.78it/s]



Fine-tune Epoch 5
Loss: 0.0466
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       108
           1       0.99      0.98      0.99       108

    accuracy                           0.99       216
   macro avg       0.99      0.99      0.99       216
weighted avg       0.99      0.99      0.99       216



In [16]:
y_true, y_pred, y_prob = evaluate(model, test_loader)

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=train_ds.classes))

print("ROC-AUC:", roc_auc_score(y_true, y_prob))


Confusion Matrix:
[[108   2]
 [  1 109]]

Classification Report:
              precision    recall  f1-score   support

      closed       0.99      0.98      0.99       110
        open       0.98      0.99      0.99       110

    accuracy                           0.99       220
   macro avg       0.99      0.99      0.99       220
weighted avg       0.99      0.99      0.99       220

ROC-AUC: 0.9998347107438017


In [17]:
torch.save(model.state_dict(), "mobilenetv3_binary.pth")
print("MobileNetV3 model saved!")


MobileNetV3 model saved!
